In [12]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
%run dataimport_cleaning.ipynb

## stacked bar chart

In [13]:
def stacked_bar_chart_plotly(main_filter, sales_filter):
    # extract and copy date from df
    df_bar = df[[main_filter,'North America', 'Europe', 'Japan', 'Others', 'Global']]
    df_bar = df_bar[df_bar['Global'] > sales_filter]

    # group by filter and sort by global sales
    df_bar_grouped = df_bar.groupby([main_filter]).sum()
    df_bar_grouped = df_bar_grouped.sort_values(by=['Global'], ascending=False)

    #main filter as column in data frame
    df_bar_grouped.reset_index(inplace=True)
    df_bar_grouped = df_bar_grouped.rename(columns = {'index':main_filter})

    # dropout Global Sales
    df_bar_grouped = df_bar_grouped [[main_filter, 'North America', 'Europe', 'Japan', 'Others']]

    fig = px.bar(df_bar_grouped, x=main_filter, y=['North America', 'Europe', 'Japan', 'Others'], color_discrete_sequence= ['#1a889d', '#4da3b3', '#80bdc9', '#b3d7de'])
    fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', title = 'number of sales (in million)')

    # legend within the graph (not beside)
    fig.update_layout(legend=dict(
    title = '  Area',
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
    return fig

## line chart

In [9]:
def line_diagram(main_filter, sales_filter):
    df_l = df[df['Global'] > sales_filter]
    df_l = df_l.groupby(['Year', main_filter], as_index=False)['Global'].sum()

    line_fig = px.line(df_l, x='Year', y='Global', color=main_filter, color_discrete_sequence= ['#015666', '#1a889d', '#4da3b3', '#80bdc9', '#b3d7de', '#cce5e9',  '#2b6b51', '#317a5c','#378a68','#50a381', '#77b89d', '#9eccb9' ])
    line_fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
    line_fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    line_fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return line_fig

## table1

In [10]:
app = Dash(__name__)

df_liste = df[['Name', 'Platform', 'Genre', 'Global']]

app.layout = html.Div([
    dash_table.DataTable(
        id='datatable_1',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df_liste.columns
        ],
        data=df_liste.to_dict('records'),
        style_cell={'textAlign': 'left'},
        editable=True,
        filter_action='native',
        sort_action='native',
        #sort_mode='multi',
        #column_selectable='single',
        #row_selectable='multi',
        #selected_columns=[],
        #selected_rows=[],
        #page_action='native',
        page_current= 0,
        page_size= 100,
        style_header={
            'backgroundColor': '#b3d7de',
            'fontWeight': 'bold'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'},
            'backgroundColor': '#ecf5f1'}]
    ),
    html.Div(id='datatable-interactivity-container')
])

@app.callback(
    Output('datatable_1', 'style_data_conditional'),
    Input('datatable_1', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


if __name__ == '__main__':
    app.run_server(debug=False, port = 8070)

NameError: name 'df' is not defined

## gauge chart

In [11]:

def gauge_chart(main_filter, sales_filter):
    regions = 'North America', 'Europe', 'Japan', 'Others'
    for region in regions:
        df_gauge = df[[main_filter,'North America', 'Europe', 'Japan', 'Others', 'Global']]
        df_gauge = df_gauge[df_gauge['Global'] > sales_filter]

        #calculate the market share
        marktanteil_NA_Sales = round(df_gauge[region].sum() / df_gauge['Global'].sum()*100, 1)

        #Gauge Chart
        fig_gaug = go.Figure(go.Indicator(
            domain = {'x': [0, 1], 'y': [0, 1]},
            value = marktanteil_NA_Sales,
            number=  {'suffix': '%' },
            mode = 'gauge+number',
            title = {'text': region, 'font': { 'size': 42 }},
            gauge = {'axis': {'range': [None, 50]},
                     'bar': {'color': '#378a68'},
                     'steps' : [
                         {'range': [0, 50], 'color': '#b3d7de'},
                     ],
                         #{'range': [50/3, 50/3*2], 'color': "blue"},
                         #{'range': [50/3*2, 50], 'color': "green"}],
                     }))        # 'threshold' : {'line': {'color': "red", 'width': 4},'thickness': 0.75, 'value': 0.09}

        return fig_gaug